<a href="https://colab.research.google.com/github/rtnaka/ai_quest_assessment/blob/main/AI_Quest2021_%E3%82%A2%E3%82%BB%E3%82%B9%E3%83%A1%E3%83%B3%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ライブラリのインストール

In [1]:
!pip install category_encoders
!pip install japanize-matplotlib
!pip install pandas_profiling
!pip install git+https://github.com/pfnet-research/xfeat.git

     |████████████████████████████████| 80 kB 3.2 MB/s 
     |████████████████████████████████| 4.1 MB 4.3 MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120274 sha256=969bb9e7922d242e88dd28eddb57ed673e2bef68fa03c4558ae23a8007ea4c25
  Stored in directory: /root/.cache/pip/wheels/83/97/6b/e9e0cde099cc40f972b8dd23367308f7705ae06cd6d4714658
Successfully built japanize-matplotlib
  Cloning https://github.com/pfnet-research/xfeat.git to /tmp/pip-req-build-iahfpzmi
  Running command git clone -q https://github.com/pfnet-research/xfeat.git /tmp/pip-req-build-iahfpzmi
     |████████████████████████████████| 302 kB 4.2 MB/s 
     |████████████████████████████████| 164 kB 43.1 MB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 
     |████████████████████████████████| 75 kB 4.3 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 141 kB 47.0 MB/s 
     |████████████████████████████

## ライブラリのインポート

In [2]:
import re
import datetime
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
from scipy import stats
import sklearn
from tqdm.notebook import tqdm  as tqdm
from matplotlib import pyplot as plt
%matplotlib inline
import japanize_matplotlib 

#EDA
from pandas_profiling import ProfileReport

#Feature Engneering
from sklearn.preprocessing import QuantileTransformer,StandardScaler
from category_encoders import OrdinalEncoder, OneHotEncoder, TargetEncoder , CountEncoder
from xfeat import SelectCategorical, LabelEncoder, Pipeline, ConcatCombination, SelectNumerical, ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer
from sklearn.feature_extraction.text import TfidfVectorizer

#Modeling
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import roc_auc_score, mean_squared_error, mean_squared_log_error, log_loss, roc_curve, confusion_matrix, plot_roc_curve
import lightgbm as lgb
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## データの読み込み

In [3]:
# Google DriveをColabにマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#最大表示行数指定
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)

In [5]:
# ファイルパス
basepath = '/content/drive/MyDrive/AI Quest2021 アセスメント/'

# 学習データ
df_train = pd.read_csv(basepath+'train.csv', index_col=0,parse_dates=["first_review", "host_since","last_review"])

# テストデータ
df_test = pd.read_csv(basepath+'test.csv', index_col=0,parse_dates=["first_review", "host_since","last_review"])

# 提出フォーマット
submission = pd.read_csv(basepath+'sample_submit.csv', index_col=0,header=None,names = ["","y"])

## 探索的データ解析

#### 行数と列数の確認　.shape

In [6]:
# DataFrameの行数と列数の確認
print("学習データ",df_train.shape)
print("テストデータ",df_test.shape)
print("提出用サンプル",submission.shape)

学習データ (55583, 28)
テストデータ (18528, 27)
提出用サンプル (18528, 1)


#### テーブルデータの一部を確認

In [7]:
# 先頭行の確認 df_train
df_train.head()

,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode,y
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",2.0,Real Bed,1.0,4.0,flexible,LA,t,My place is meant for family and a few friends...,2016-07-27,t,f,NaN,2016-07-13,f,2016-07-27,33.788931,-118.154761,The Penthouse,NaN,1,Apartment,60.0,Private room,NaN,90804,138.0
1,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,DC,t,This is a new listing for a lovely guest bedro...,2016-09-12,t,t,100%,2015-12-30,f,2017-03-31,38.934810,-76.978190,Guest Bedroom in Brookland,Brookland,9,House,100.0,Private room,https://a0.muscache.com/im/pictures/e4d8b51f-6...,20018,42.0
2,2,"{TV,Internet,""Wireless Internet"",Kitchen,""Indo...",2.0,Real Bed,1.0,1.0,strict,NYC,t,We're looking forward to your stay at our apt....,2016-06-15,t,f,100%,2016-05-21,t,2017-08-13,40.695118,-73.926240,Clean Modern Room in Lux Apt 1 Block From J Train,Bushwick,27,Apartment,83.0,Private room,https://a0.muscache.com/im/pictures/5ffecc9b-d...,NaN,65.0
3,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,SF,t,BEST CITY VIEWS - - ROOF DECK W/ BBQ & WiFi - ...,2014-03-15,t,t,100%,2012-06-19,t,2017-09-03,37.796728,-122.411906,BEST views + reviews! 5/5 stars*****,Nob Hill,38,Apartment,95.0,Private room,NaN,94133,166.0
4,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,Real Bed,1.0,1.0,strict,NYC,t,Charming Apartment on the upper west side of M...,2015-08-05,t,t,100%,2015-03-25,f,2017-09-10,40.785050,-73.974691,Charming 1-bedroom - UWS Manhattan,Upper West Side,5,Apartment,100.0,Entire home/apt,https://a0.muscache.com/im/pictures/92879730/5...,10024,165.0


In [8]:
# 先頭行の確認 df_test
df_test.head()

,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode
id,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",2.0,Real Bed,2.0,2.0,strict,Boston,t,Feel free to book INSTANTLY. You can check-in ...,2017-01-09,t,f,100%,2016-08-23,t,2017-09-25,42.359278,-71.069962,Gorgeous 2BR/2BA Duplex in Beacon Hill,Beacon Hill,58,House,90.0,Entire home/apt,https://a0.muscache.com/im/pictures/7e4808b4-5...,02114
1,3,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,moderate,LA,t,The guest house is close to: Equinox West Holl...,2016-08-17,t,t,100%,2014-09-03,f,2017-05-02,34.084747,-118.367355,Luxury 1 Bedroom West Hollywood City Center,West Hollywood,4,Guesthouse,100.0,Entire home/apt,https://a0.muscache.com/im/pictures/5392fbd6-6...,90046
2,2,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",1.0,Real Bed,0.0,1.0,flexible,NYC,f,Private room in a three bedroom apartment in N...,NaT,t,t,100%,2012-10-17,f,NaT,40.720541,-73.959192,Bedroom with Patio in Prime Williamsburg Locat...,Williamsburg,0,Apartment,NaN,Private room,https://a0.muscache.com/im/pictures/544d3b89-d...,11249.0
3,4,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,2.0,strict,NYC,f,The apartment is located in historic Bed Stuy ...,NaT,t,t,NaN,2013-01-23,f,NaT,40.681117,-73.944091,Cozy apartment in Brooklyn,Bedford-Stuyvesant,0,Apartment,NaN,Entire home/apt,https://a0.muscache.com/im/pictures/26baf7ba-0...,11216
4,3,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.5,Real Bed,1.0,2.0,strict,LA,t,"Our cozy, pet friendly one bedroom apartment/l...",2015-08-01,t,t,100%,2014-12-28,f,2016-09-11,34.150995,-118.409359,"Cozy, sunny, pet friendly loft/apt",NaN,6,Loft,92.0,Entire home/apt,https://a0.muscache.com/im/pictures/86107545/9...,91604


In [9]:
# 先頭行の確認 submission
submission.head()

,y
,
0,10
1,10
2,10
3,10
4,10


#### 特徴量（カラム）のデータ型の確認

In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55583 entries, 0 to 55582
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   accommodates            55583 non-null  int64         
 1   amenities               55583 non-null  object        
 2   bathrooms               55436 non-null  float64       
 3   bed_type                55583 non-null  object        
 4   bedrooms                55512 non-null  float64       
 5   beds                    55487 non-null  float64       
 6   cancellation_policy     55583 non-null  object        
 7   city                    55583 non-null  object        
 8   cleaning_fee            55583 non-null  object        
 9   description             55583 non-null  object        
 10  first_review            43675 non-null  datetime64[ns]
 11  host_has_profile_pic    55435 non-null  object        
 12  host_identity_verified  55435 non-null  object

In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18528 entries, 0 to 18527
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   accommodates            18528 non-null  int64         
 1   amenities               18528 non-null  object        
 2   bathrooms               18475 non-null  float64       
 3   bed_type                18528 non-null  object        
 4   bedrooms                18508 non-null  float64       
 5   beds                    18493 non-null  float64       
 6   cancellation_policy     18528 non-null  object        
 7   city                    18528 non-null  object        
 8   cleaning_fee            18528 non-null  object        
 9   description             18528 non-null  object        
 10  first_review            14572 non-null  datetime64[ns]
 11  host_has_profile_pic    18488 non-null  object        
 12  host_identity_verified  18488 non-null  object

#### 欠損・ユニーク数の確認

In [ ]:
# 欠損確認
total_nan = df_train.isnull().sum().sort_values(ascending=False)
percent_nan = (df_train.isnull().sum()/len(df_train)*100).sort_values(ascending=False)
df_nan = pd.concat([total_nan, percent_nan], axis=1, keys=['NaN合計', 'NaN 割合[%]'])

# ユニーク数確認
nunique = df_train.nunique().sort_values(ascending=False)
percent_unique = df_train.nunique().sort_values(ascending=False)/len(df_train)*100
df_nunique = pd.concat([nunique,percent_unique],axis=1,keys=["ユニーク数","ユニーク数 割合[%]"])

df_nanunique = pd.concat([df_nan,df_nunique],axis=1)
df_uniquenan = pd.concat([df_nunique,df_nan],axis=1)

In [ ]:
df_nanunique.head(len(df_train))

In [ ]:
# 欠損確認
total_nan = df_test.isnull().sum().sort_values(ascending=False)
percent_nan = (df_test.isnull().sum()/len(df_test)*100).sort_values(ascending=False)
df_nan = pd.concat([total_nan, percent_nan], axis=1, keys=['NaN合計', 'NaN 割合[%]'])

# ユニーク数確認
nunique = df_test.nunique().sort_values(ascending=False)
percent_unique = df_test.nunique().sort_values(ascending=False)/len(df_train)*100
df_nunique = pd.concat([nunique,percent_unique],axis=1,keys=["ユニーク数","ユニーク数 割合[%]"])

df_nanunique = pd.concat([df_nan,df_nunique],axis=1)
df_uniquenan = pd.concat([df_nunique,df_nan],axis=1)

In [ ]:
df_nanunique.head(len(df_train))

#### 基本統計量　（数値特徴量）

In [ ]:
pd.options.display.float_format = '{:.7g}'.format

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
# １7０0$以上の物件について中身を確認
df_train[(df_train["y"] > 1500)&(df_train["accommodates"] <3) ]

In [ ]:
# 10$以下の物件について中身を確認
df_train[df_train["y"] < 10 ]

In [ ]:
delnums = [14807,39193,40313,40732,29385,34730]
for delnum in delnums :
  df_train.drop(delnum,inplace=True)

#### 数値特徴量のヒストグラムを確認

In [ ]:
# dtypeが数値のカラム名

numerical_feats = df_train.dtypes[df_train.dtypes != "object"].index
numerical_feats

In [ ]:
sns.displot(data=df_train,x="review_scores_rating")

#### 各特徴量を個別に確認

In [ ]:
# dtypeがobject（数値でないもの）のカラム名とユニーク数、欠損数の確認
# カテゴリ値のカラム確認
categorical_feats = df_train.dtypes[df_train.dtypes == "object"].index
categorical_feats

In [ ]:
# テキスト特徴量
col = "amenities"
df_train[col].unique()

In [ ]:
# カテゴリ　
col = "bed_type"
target = "y"

plt.figure(figsize=(20, 10))
sns.violinplot(data=df_train,x=col,y=target)
pd.DataFrame(df_train[col].value_counts().sort_values(ascending = False))

In [ ]:
col = "cancellation_policy"
plt.figure(figsize=(20, 10))
sns.violinplot(data=df_train,x=col,y=target)

pd.DataFrame(df_train[col].value_counts().sort_values(ascending = False))

In [ ]:
col = "city"

target = "y"
plt.figure(figsize=(20, 10))
sns.violinplot(data=df_train,x=col,y=target)

pd.DataFrame(df_test[col].value_counts().sort_values(ascending = False))

In [ ]:
col = "first_review"
pd.DataFrame(df_train[col].dt.year.value_counts())

In [ ]:
pd.DataFrame(df_test[col].dt.year.value_counts())

In [ ]:
df_train["y"].groupby(df_train[col].dt.year).mean()

In [ ]:
col = "host_has_profile_pic"
pd.DataFrame(df_train[col].value_counts().sort_values(ascending = False))

In [ ]:
pd.DataFrame(df_test[col].value_counts().sort_values(ascending = False))

In [ ]:
col = "host_identity_verified"
pd.DataFrame(df_train[col].value_counts().sort_values(ascending = False))

In [ ]:
pd.DataFrame(df_test[col].value_counts().sort_values(ascending = False))

In [ ]:
col = "host_response_rate"
pd.DataFrame(df_train[col].value_counts().sort_values(ascending = False))

In [ ]:
df_train[col].unique()

In [ ]:
col = "host_since"
pd.DataFrame(df_train[col].value_counts().sort_values(ascending = False))

In [ ]:
df_train["y"].groupby(df_train[col].dt.year).mean()

In [ ]:
col = "instant_bookable"
pd.DataFrame(df_train[col].value_counts().sort_values(ascending = False))

In [ ]:
col = "property_type"
pd.DataFrame(df_train[col].value_counts().sort_values(ascending = False))

## 特徴量エンジニアリング（Feature Engineering）

ture/false →　1/0 に変換
- cleaning_fee
- host_has_profile_pic	
- host_identity_verified
- instant_bookable

%を削除し数値変換
- host_response_rate

列削除
- latitude	longitude

欠損有無のみ列として残す
- thumbnail_url

桁数を減らして利用
- zipcode

2017/12/31を基準日とし、差分を取る
- first_review,last_review,host_since

#### Xとyに分割（X_train,y_train,X_testの生成）

In [ ]:
# Xとyに分割 (X_train,y_train,X_test の生成)
target = "y"
y_train = df_train[target]
X_train = df_train.drop([target], axis=1)
X_test = df_test.copy()

#### 欠損数の追加

In [ ]:
#NaNに関数特徴量を新たに作成
for col in X_train.columns:
    if X_train[col].isnull().any():
        X_train[col + '_isNaN'] = X_train[col].isnull().astype(int)
        X_test[col + '_isNaN'] = X_test[col].isnull().astype(int)   

X_train['null_count'] = X_train.isnull().sum(axis=1)
X_test['null_count'] = X_test.isnull().sum(axis=1)

#### ターゲットの対数を取り分布を正規分布に近づける

In [ ]:
y_train = y_train.apply(np.log1p)

#### 列削除

In [ ]:
cols = ["thumbnail_url","latitude","longitude"]
X_train.drop(cols,axis = 1,inplace=True)
X_test.drop(cols,axis = 1,inplace=True)

#### ture/false →　1/0 に変換

In [ ]:
cols = ["cleaning_fee","host_has_profile_pic","host_identity_verified","instant_bookable"]

for col in cols :
  X_train[col] = X_train[col].apply(lambda x: 1 if x == "t" else 0 )
  X_test[col] = X_test[col].apply(lambda x: 1 if x == "t" else 0)

#### %を削除し数値変換

In [ ]:
col = "host_response_rate"
X_train[col].fillna("-1%",inplace=True)
X_test[col].fillna("-1%",inplace=True)
X_train[col] = X_train[col].str.strip('%').astype(float)
X_test[col] = X_test[col].str.strip('%').astype(float)

#### 桁数を減らして利用

In [ ]:
col = "zipcode"
X_train[col].fillna("nan",inplace=True)
X_test[col].fillna("nan",inplace=True)
X_train[col] = X_train[col].apply(lambda x:x[:3])
X_test[col] = X_test[col].apply(lambda x:x[:3])

#### 2017/12/31を基準日とし、差分を取る

In [ ]:
col = "last_review"
day = datetime.datetime(2017, 10, 31, 0, 0, 0)
X_train[col].fillna(day,inplace=True)
X_test[col].fillna(day,inplace=True)
X_train[col] =  X_train[col].apply(lambda x: day - x).dt.days
X_test[col] =  X_test[col].apply(lambda x: day - x).dt.days

In [ ]:
cols = ["first_review","host_since"]

for col in cols:
  day = datetime.datetime(1,1,1, 0, 0, 0)
  X_train[col] =  X_train[col].apply(lambda x: str(x.year *100 + x.month))
  X_test[col] =  X_test[col].apply(lambda x: str(x.year *100 + x.month))

#### 欠損埋め

In [ ]:
pd.DataFrame(X_train.isnull().sum().sort_values(ascending = False)).head(7)

In [ ]:
cols = ["beds","bedrooms","bathrooms"]
for col in cols:
  X_train[col].fillna(X_train[col].median(),inplace=True)
  X_test[col].fillna(X_test[col].median(),inplace=True)

In [ ]:
col = "review_scores_rating"
X_train[col].fillna(-1,inplace=True)
X_test[col].fillna(-1,inplace=True)

col = "neighbourhood"
X_train[col].fillna("nan",inplace=True)
X_test[col].fillna("nan",inplace=True)

In [ ]:
pd.DataFrame(X_train.isnull().sum().sort_values(ascending = False)).head()

In [ ]:
X_train.head()

### 数値特徴量

In [ ]:
# dtypeが数値のカラム名とユニーク数、欠損数の確認
# 数値のカラム確認

numerical_feats = X_train.dtypes[X_train.dtypes != "object"].index
numerical_feats

In [ ]:
# 数値特徴量２つの乗算を実施
encoder_num2 = Pipeline(
    [
        SelectNumerical(),
        ArithmeticCombinations(
            input_cols=['accommodates', 'bathrooms', 'bedrooms', 'beds'], 
            drop_origin=True, 
            operator="*", 
            r=2,
        ),
    ]
)

combi_num2_train = encoder_num2.fit_transform(X_train)
combi_num2_test = encoder_num2.fit_transform(X_test)

X_train = pd.concat([X_train,combi_num2_train],axis=1)
X_test = pd.concat([X_test,combi_num2_test],axis=1)

In [ ]:
# 数値特徴量２つの乗算を実施
encoder_num2 = Pipeline(
    [
        SelectNumerical(),
        ArithmeticCombinations(
            input_cols=[ 'number_of_reviews','review_scores_rating'], 
            drop_origin=True, 
            operator="*", 
            r=2,
        ),
    ]
)

combi_num2_train = encoder_num2.fit_transform(X_train)
combi_num2_test = encoder_num2.fit_transform(X_test)

X_train = pd.concat([X_train,combi_num2_train],axis=1)
X_test = pd.concat([X_test,combi_num2_test],axis=1)

In [ ]:
# 数値特徴量　host
encoder_num19 = Pipeline(
    [
        SelectNumerical(),
        ArithmeticCombinations(
            input_cols=['host_has_profile_pic', 'host_identity_verified', 'host_response_rate'], 
            drop_origin=True, 
            operator="+", 
            r=3,
        ),
    ]
)

combi_num19_train = encoder_num19.fit_transform(X_train)
combi_num19_test = encoder_num19.fit_transform(X_test)

X_train = pd.concat([X_train,combi_num19_train],axis=1)
X_test = pd.concat([X_test,combi_num19_test],axis=1)

In [ ]:
X_train.head()

### カテゴリ特徴量

In [ ]:
# dtypeがobject（数値でないもの）
categorical_feats = X_train.dtypes[X_train.dtypes == "object"].index
categorical_feats = categorical_feats.drop(['amenities','description'])
categorical_feats

In [ ]:
#カテゴリ変数の組み合わせを作成（組み合わせ数＝２）
encoder_str2 = Pipeline(
    [
      SelectCategorical(exclude_cols=[ 'first_review', 'host_since','name' ]),
      ConcatCombination(drop_origin=True, output_suffix="", r=2)
    ]
  )

combi_str2_train = encoder_str2.fit_transform(X_train)
combi_str2_test =  encoder_str2.fit_transform(X_test)
X_train = pd.concat([X_train,combi_str2_train],axis=1)
X_test = pd.concat([X_test,combi_str2_test],axis=1)

In [ ]:
# dtypeがobject（数値でないもの）
categorical_feats = X_train.dtypes[X_train.dtypes == "object"].index
categorical_feats = categorical_feats.drop(['amenities','description'])
categorical_feats

In [ ]:
# # Target Encoding
# y_train_bin = pd.cut(y_train , 10, labels=False).astype(int)
# target = 'y'
# X_temp = pd.concat([X_train, y_train], axis=1)

# for col in categorical_feats:

#     # X_testはX_trainでエンコーディングする
#     summary = X_temp.groupby([col])[target].mean()
#     X_test[col] = X_test[col].map(summary) 


#     # X_trainのカテゴリ変数をoofでエンコーディングする
#     skf = StratifiedKFold(n_splits=5, random_state=71, shuffle=True)
#     enc_train = pd.Series(np.zeros(len(X_train)), index=X_train.index)

#     for i, (train_ix, val_ix) in enumerate((skf.split(X_train, y_train_bin))):
#         X_train_, _ = X_temp.iloc[train_ix], y_train.iloc[train_ix]
#         X_val, _ = X_temp.iloc[val_ix], y_train.iloc[val_ix]

#         summary = X_train_.groupby([col])[target].mean()
#         enc_train.iloc[val_ix] = X_val[col].map(summary)
        
#     X_train[col]  = enc_train

In [ ]:
# count encording
encoder = CountEncoder(cols=categorical_feats)
enctrain = encoder.fit_transform(X_train)
enctest = encoder.transform(X_test)

In [ ]:
enctrain.head()

### テキスト特徴量

In [ ]:
#大文字→小文字変換
cols = ["amenities","description"]
for col in cols:
  enctrain[col] = enctrain[col].str.lower()
  enctest[col] = enctest[col].str.lower()

In [ ]:
cols = ["amenities","description"]

enctrain_copy = enctrain.copy()
enctest_copy = enctest.copy()

enctrain.drop(cols,axis=1,inplace=True)
enctest.drop(cols,axis=1,inplace=True)


for col in cols:
  tfidf_train = TfidfVectorizer(max_features=50)
  enctrain_tfidf = tfidf_train.fit_transform(enctrain_copy[col].values.astype('U'))

  tfidf_test = TfidfVectorizer(max_features=50 , vocabulary=tfidf_train.vocabulary)
  enctest_tfidf = tfidf_test.fit_transform(enctest_copy[col].values.astype('U'))

  enctrain = sp.sparse.hstack([enctrain.values, enctrain_tfidf]).todense()
  enctrain = pd.DataFrame(enctrain)
  enctest = sp.sparse.hstack([enctest.values, enctest_tfidf]).todense()
  enctest = pd.DataFrame(enctest)

In [ ]:
enctrain.head()

## モデリング

In [ ]:
X_train = enctrain.copy()
X_test = enctest.copy()

In [ ]:
X_train.shape,X_test.shape

In [ ]:
y_train_bin = pd.cut(y_train ,10, labels=False).astype(int)

In [ ]:
def train_and_predict(x_train,y_train,x_test,seeds) :

  clf = LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
                                importance_type='split', learning_rate=0.05, max_depth=-1,
                                min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
                                n_estimators=9999, n_jobs=-1, num_leaves=15, objective='regression',
                                random_state=71, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


  scores = []
  preds = []
  
  for seed in tqdm(seeds):
    skf = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
    y_test_pred = np.zeros(len(x_test))

    for i, (train_ix, test_ix) in enumerate(skf.split(x_train, y_train_bin)):
      x_train_, y_train_ = x_train.values[train_ix], y_train.values[train_ix]
      x_val, y_val = x_train.values[test_ix], y_train.values[test_ix]
      
      clf.fit(x_train_, y_train_, early_stopping_rounds=10, eval_metric='rmse', eval_set=[(x_val, y_val)])
      y_val_pred = clf.predict(x_val)
      print('CV Split Score is %f' % (np.sqrt(mean_squared_error(y_val, y_val_pred))))
      scores.append(np.sqrt(mean_squared_error(y_val, y_val_pred)))

      y_test_pred += clf.predict(x_test)

    #CV毎のテストデータ予測結果の平均を取り、predに代入
    y_test_pred /= 5
    preds.append(y_test_pred)

    #seed毎に計算されるテストデータ予測予測結果の平均を取る
    cv_scores_ave = np.mean(scores,axis=0)

    #全CVの交差検定スコアの平均を取る
    pred = np.mean(preds, axis=0)

    #変数重要度
    feature_importance = pd.DataFrame(
                                          clf.booster_.feature_importance(importance_type='gain'), 
                                          index = x_train.columns, columns=['importance']).sort_values(['importance'], 
                                          ascending=False
                                        )

  return pred , cv_scores_ave , feature_importance

In [ ]:
seeds = [31,71,91]
pred , cv_scores_ave , feature_importance = train_and_predict(X_train,y_train,X_test,seeds)

In [ ]:
target = "y"
# submission[target] = np.expm1(pred)
submission[target] = pred

In [ ]:
submission[submission[target] < 10 ]

In [ ]:
submission.loc[9060][target] = 15

In [ ]:
submission[target] = submission[target].astype(int)

In [ ]:
submission.describe()

In [ ]:
df_test.loc[9060]

In [ ]:
df_train[(df_train["beds"] > 7)&(df_train["property_type"] == "Hostel")].head()

In [ ]:
submission.to_csv(basepath+'submit_02.csv',header=None)

In [ ]:
len(feature_importance)

In [ ]:
use_col  = feature_importance.index[:100]
seeds = [31,71,91]
pred , cv_scores_ave , feature_importance = train_and_predict(X_train[use_col],y_train,X_test[use_col],seeds)

In [ ]:
figure = plt.figure(figsize=(25, 15))
sns.barplot(x="importance", y=feature_importance.index, data=feature_importance)

In [ ]:
target = "y"
submission[target] = pred
submission.describe()

In [ ]:
submission[submission[target] < 10 ]

In [ ]:
submission.loc[9060][target] = 15

In [ ]:
submission[target] = submission[target].astype(int)

In [ ]:
submission.describe()

In [ ]:
submission.to_csv(basepath+'submit_03.csv',header=None)

In [ ]:
submission
